In [ ]:
import pandas as pd
import numpy as np

melbourne = pd.read_csv("temperature_Melbourne_celsius.csv", delimiter=";")
oslo = pd.read_csv("temperature_Oslo_celsius.csv", delimiter=";")
quito = pd.read_csv("temperature_Quito_celsius.csv", delimiter=";")

# Ejercicio 1A

In [34]:
# Asegurar que la columna de temperatura sea la correcta
columna_temperatura = "AvgTemperature"  # Cambia esto según el nombre de la columna en los archivos

# Calcular la media y desviación estándar
media_melbourne = melbourne[columna_temperatura].mean()
media_oslo = oslo[columna_temperatura].mean()
media_quito = quito[columna_temperatura].mean()

desvio_melbourne = melbourne[columna_temperatura].std()
desvio_oslo = oslo[columna_temperatura].std()
desvio_quito = quito[columna_temperatura].std()

# Mostrar los resultados con formato mejorado
print(f"Media de Melbourne: {media_melbourne:.2f}°C")
print(f"Media de Oslo: {media_oslo:.2f}°C")
print(f"Media de Quito: {media_quito:.2f}°C")

print(f"Desvío estándar de Melbourne: {desvio_melbourne:.2f}°C")
print(f"Desvío estándar de Oslo: {desvio_oslo:.2f}°C")
print(f"Desvío estándar de Quito: {desvio_quito:.2f}°C")

Media de Melbourne: 17.53°C
Media de Oslo: 4.61°C
Media de Quito: 13.60°C
Desvío estándar de Melbourne: 6.48°C
Desvío estándar de Oslo: 9.46°C
Desvío estándar de Quito: 1.30°C


### 1 -B

In [35]:
from scipy.signal import correlate
import numpy as np

x = oslo[columna_temperatura].values
y = melbourne[columna_temperatura].values

corr = correlate(x, y, mode='full')
print(f"Correlación entre Oslo y Melbourne: {corr.max():.2f}")

x = oslo[columna_temperatura].values
y = quito[columna_temperatura].values

corr = correlate(x, y, mode='full')
print(f"Correlación entre Oslo y Quito: {corr.max():.2f}")

x = quito[columna_temperatura].values
y = melbourne[columna_temperatura].values

corr = correlate(x, y, mode='full')
print(f"Correlación entre quito y Melbourne: {corr.max():.2f}")


Correlación entre Oslo y Melbourne: 624455.00
Correlación entre Oslo y Quito: 376918.00
Correlación entre quito y Melbourne: 1393015.00


Analizando los resultados obtenidos observamos que entre oslo y cualquir otra ciudad no hay tremperatura climatica similar, en cambio entre Quito y Merburne hay una mayor correlacion por lo que hay una similitud, lo cual tiene sentido observando las medias.

## 2.
### Considerando los valores de temperatura t que componen cada señal Si, construir una nueva señal Ti  compuesta por una secuencia de símbolos discretos F, T o C, definidos según: 

In [36]:

def clasificar_temperatura(t):
    if t < 11:
        return 'F'
    elif t < 19:
        return 'T'
    else:
        return 'C'

for df in [oslo, quito, melbourne]:
    df['Clima'] = df[columna_temperatura].apply(clasificar_temperatura)



#### Parte 1

In [37]:
def contar_climas(df):
    conteo = df['Clima'].value_counts()
    f =  conteo.get('F', 0) 
    t = conteo.get('T', 0)
    c =  conteo.get('C', 0)
    return f, t, c

f_oslo, t_oslo, c_oslo = contar_climas(oslo)
f_quito, t_quito, c_quito = contar_climas(quito)
f_melbourne, t_melbourne, c_melbourne = contar_climas(melbourne)

def contar_transiciones(df, origen):
    """
    Cuenta cuántas veces después de 'origen' aparece cada letra ('F', 'T', 'C') en la columna 'Clima' del DataFrame.
    Retorna un diccionario con el conteo de transiciones.
    """
    clima = df['Clima'].values
    transiciones = {'F': 0, 'T': 0, 'C': 0}
    for i in range(len(clima) - 1):
        if clima[i] == origen:
            siguiente = clima[i + 1]
            if siguiente in transiciones:
                transiciones[siguiente] += 1
    return transiciones

def distribucion_probabilidad(df,f, t, c):
    matriz = [[0,0,0],
            [0,0,0],
            [0,0,0]]
    i = 0
    for l in ['F', 'T', 'C']:
        transiciones = contar_transiciones(df, l)
        matriz[i][0] = round(transiciones['F']/f, 3)
        matriz[i][1] = round(transiciones['T']/t, 3)
        matriz[i][2] = round(transiciones['C']/c, 3)
        i += 1
    return np.array(matriz)   

distribucion_oslo = distribucion_probabilidad(oslo, f_oslo, t_oslo, c_oslo)
distribucion_quito = distribucion_probabilidad(quito, f_quito, t_quito, c_quito)
distribucion_melbourne = distribucion_probabilidad(melbourne, f_melbourne, t_melbourne, c_melbourne)

print("Distribución de probabilidad de Oslo:")
print(distribucion_oslo)
print("--------------------------------------------")
print("Distribución de probabilidad de Quito:")
print(distribucion_quito)
print("--------------------------------------------")
print("Distribución de probabilidad de Melbourne:")
print(distribucion_melbourne)


Distribución de probabilidad de Oslo:
[[0.959 0.099 0.   ]
 [0.041 0.865 0.296]
 [0.    0.035 0.704]]
--------------------------------------------
Distribución de probabilidad de Quito:
[[0.054 0.006 0.   ]
 [0.946 0.993 1.   ]
 [0.    0.001 0.   ]]
--------------------------------------------
Distribución de probabilidad de Melbourne:
[[0.47  0.033 0.003]
 [0.495 0.836 0.144]
 [0.035 0.131 0.853]]


#### Parte 2

In [ ]:
def contarColumnas(df):
    conteo = df['Clima'].value_counts()
    f =  conteo.get('F', 0) 
    t = conteo.get('T', 0)
    c =  conteo.get('C', 0)
    return f+t+c
def distribucion_probabilistica_estacionaria(df):
    totalColumnas=contarColumnas(df)
    f,t,c = contar_climas(df)
    res = [f /totalColumnas, t / totalColumnas, c / totalColumnas]

    return np.array(res, dtype=np.float64)

distribucion_estacionaria_oslo = distribucion_probabilistica_estacionaria(oslo)
print(distribucion_estacionaria_oslo)

[0.68583162 0.28062971 0.03353867]
1.0


In [153]:
def matriz_acumulada(matriz):
    res = matriz.copy()
    for i in range(0,len(matriz)): #i recorre las filas
        for j in range(1, len(matriz[i])): # j recorre las columnas
            res[j][i] += res[j-1][i]
    return res

acumulada_oslo = matriz_acumulada(distribucion_oslo)
acumulada_quito = matriz_acumulada(distribucion_quito)
acumulada_melbourne = matriz_acumulada(distribucion_melbourne)

print("Matriz acumulada de Oslo:")
print(acumulada_oslo)

Matriz acumulada de Oslo:
[[0.959 0.099 0.   ]
 [1.    0.964 0.296]
 [1.    0.999 1.   ]]


In [152]:
import random

def Media_recurrencia(simbolo, matriz):
    retornos= 0        # cantidad de retornos a si 
    media= 0          #media recurrencia actual    
    media_ant= -1  # media recurrencia anterior    
    t_actual= 0   
    s= simbolo
    while  not np.allclose(media, media_ant, atol=1e-3) or (t_actual<100) :
        s= Sig_dado_Ant (s, matriz)
        t_actual+=1
        if ( s == simbolo ): #hay retorno 
            retornos+=1
            media_ant = media
            media= t_actual/retornos
    return media

def Sig_dado_Ant(simbolo, matriz):
    r = random.random()
    if simbolo == "F":
        simbolo = 0
    elif simbolo == "T":
        simbolo = 1
    else:
        simbolo = 2

    for i in range(3):
        if ( r < matriz[i, simbolo]):
            if(i==0):
                return "F"
            elif(i==1):
                return "T"
            else:
                return "C"

MR_F_oslo = Media_recurrencia("F", acumulada_oslo)
MR_T_oslo = Media_recurrencia("T", acumulada_oslo)
MR_C_oslo = Media_recurrencia("C", acumulada_oslo)
"""
MR_F_quito = Media_recurrencia("F", acumulada_quito)
MR_T_quito = Media_recurrencia("T", acumulada_quito)
MR_C_quito = Media_recurrencia("C", acumulada_quito)
"""
MR_F_melbourne = Media_recurrencia("F", acumulada_melbourne)
MR_T_melbourne = Media_recurrencia("T", acumulada_melbourne)
MR_C_melbourne = Media_recurrencia("C", acumulada_melbourne)

print(f"Media de recurrencia de F en Oslo: {MR_F_oslo:.2f}")
print(f"Media de recurrencia de T en Oslo: {MR_T_oslo:.2f}")
print(f"Media de recurrencia de C en Oslo: {MR_C_oslo:.2f}")
"""
print("--------------------------------------------")

print(f"Media de recurrencia de F en Quito: {MR_F_quito:.2f}")
print(f"Media de recurrencia de T en Quito: {MR_T_quito:.2f}")
print(f"Media de recurrencia de C en Quito: {MR_C_quito:.2f}")
"""
print("--------------------------------------------")
print(f"Media de recurrencia de F en Melbourne: {MR_F_melbourne:.2f}")
print(f"Media de recurrencia de T en Melbourne: {MR_T_melbourne:.2f}")
print(f"Media de recurrencia de C en Melbourne: {MR_C_melbourne:.2f}")


Media de recurrencia de F en Oslo: 4.02
Media de recurrencia de T en Oslo: 3.97
Media de recurrencia de C en Oslo: 1.00
--------------------------------------------
Media de recurrencia de F en Melbourne: 26.24
Media de recurrencia de T en Melbourne: 2.08
Media de recurrencia de C en Melbourne: 2.17
